# Toy Radix Benders Multicut

In [1]:
LP_RELAXATION = True
PREVENT_ZERO = True
REG_WEIGHT = 0. #1e-4    # Regularization weight

In [2]:
%load_ext line_profiler

In [3]:
from gurobipy import *

import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams['svg.fonttype'] = 'none'
pd.set_option('display.max_colwidth', -1)
%matplotlib inline

from dynamicme.decomposition import Decomposer
from dynamicme.callback_gurobi import cb_benders
from dynamicme.optimize import Optimizer, StackOptimizer
from dynamicme.optimize import Constraint, Variable

from cobra.io import load_json_model
from cobra import Metabolite, Reaction
from six import iteritems

import numpy as np
import cobra

(<type 'exceptions.ImportError'>, ImportError('No module named cplex',), <traceback object at 0x7fc0862e4758>)


### Try optimizing using radix for one condition first

In [4]:
#----------------------------------------
# Starting from basal model
ijomc = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
mdl_ref = ijomc

In [5]:
ijomc.optimize()
mu_crowd0 = ijomc.reactions.BIOMASS_Ecoli_core_w_GAM.x
print(mu_crowd0)

0.873921506968


In [6]:
df_meas = pd.read_csv('/home/laurence/ME/data/dynamicME/beg/growth_meas.csv')

ex_rxns = [r for r in df_meas.ex_rxn.unique() if mdl_ref.reactions.has_id(r)]
df_meas = df_meas[ df_meas.ex_rxn.isin(ex_rxns)]
conds = df_meas.substrate.unique()

# N_CONDS = len(conds)
N_CONDS = 2

df_conds = pd.DataFrame([{'cond':r['substrate'], 'rxn':ex_rxn, 'lb':-10 if r['ex_rxn']==ex_rxn else 0, 'ub':1000., 'obj':0.} for i,r in df_meas.iterrows() for ex_rxn in ex_rxns])

if N_CONDS<=3:
    df_conds = df_conds[ df_conds.cond.isin(['glucose','acetate','succinate'][0:N_CONDS])]
else:
    df_conds = df_conds[ df_conds.cond.isin(conds[0:N_CONDS])]

## Need to allow higher growth for acetate with higher uptake rate

In [7]:
df_conds.loc[ (df_conds.cond=='acetate') & (df_conds.rxn=='EX_ac_e'), 'lb'] = -20

In [8]:
from dynamicme.decomposition import BendersSubmodel, BendersMaster
from dynamicme.generate import copy_model

In [9]:
from dynamicme.optimize import Variable, Constraint
import numpy as np

radix = 2.
print('Radix:',radix)
#powers = np.arange(-3,4)
powers = [-1, 0, 1]
print('Powers:', powers)
digits_per_power = radix
pwr_max = max(powers)
digits = list(set(np.linspace(1, radix-1, digits_per_power)))
print('Digits:', digits)

# Get the group ID from reference model
mu_id = 'BIOMASS_Ecoli_core_w_GAM'
mdl_ref = ijomc
crowding_ref = mdl_ref.metabolites.crowding
conds = df_conds.cond.unique()
sub_dict = {}
for cond in conds:
    mdl_ind = cond
    mdl = copy_model(ijomc, suffix='_%s'%mdl_ind)            
    opt = Optimizer(mdl)
    gap = opt.add_duality_gap_constraint(INF=1e3, inplace=True, index=mdl_ind)
    #----------------------------------------------------
    # Now, add min abs err
    #----------------------------------------------------
    for rxn in gap.reactions:
        rxn.objective_coefficient = 0.
    mu_meas = df_meas[ df_meas.substrate==mdl_ind].growth_rate_1_h.iloc[0]
    sp = Variable('sp_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sn = Variable('sn_%s'%mdl_ind, lower_bound=0., upper_bound=1e3)
    sp.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    sn.objective_coefficient = (1.-REG_WEIGHT)/(mu_meas+1) #1.
    cons = Constraint('abs_err_%s'%mdl_ind)
    cons._constraint_sense = 'E'
    cons._bound = mu_meas
    gap.add_metabolites(cons)
    gap.add_reactions([sp,sn])
    # mu - mu_meas = sp-sn
    # mu -sp + sn = mu_meas
    # min sp + sn
    sp.add_metabolites({cons:-1.})
    sn.add_metabolites({cons:1.})
    rxn_mu = gap.reactions.get_by_id(mu_id+'_%s'%mdl_ind)
    rxn_mu.add_metabolites({cons:1.})
    #----------------------------------------------------
    
    dfi = df_conds[ df_conds.cond==cond]
    var_cons_dict = {}
    for rxn_ref in crowding_ref.reactions:    
        crowding_p = mdl.metabolites.get_by_id('crowding_%s'%mdl_ind)
        var_d = mdl.reactions.get_by_id('wa_%s'%crowding_p.id)
        rxn_p = mdl.reactions.get_by_id(rxn_ref.id+'_%s'%mdl_ind)
        cons_ds = [m for m in var_d.metabolites.keys() if rxn_p.id==m.id]        
        a0 = rxn_p.metabolites[crowding_p]
        if var_cons_dict.has_key(rxn_ref.id):
            var_cons_dict[rxn_ref.id] += [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        else:
            var_cons_dict[rxn_ref.id] = [(rxn_p, crowding_p, a0)] + [(var_d, cons_d, a0) for cons_d in cons_ds]
        
    opt.to_radix(gap, var_cons_dict, radix, powers, digits=digits, prevent_zero=PREVENT_ZERO)    
    sub = BendersSubmodel(gap, cond)
    sub_dict[cond] = sub
    
for group_id in var_cons_dict.keys():
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):
            yid = 'binary_%s%s%s'%(group_id,k,l)
            y   = gap.reactions.get_by_id(yid)
            ### PREFER pwr=0, digit=1
            if pwr==0 and digit==1:
                y.objective_coefficient = 0.
            else:
                y.objective_coefficient = REG_WEIGHT
    
master = BendersMaster(gap)

('Radix:', 2.0)
('Powers:', [-1, 0, 1])
('Digits:', [1.0])
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter InfUnbdInfo to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter LazyConstraints to 1
   Prev: 0  Min: 0  Max: 1  Default: 0
Changed value of parameter IntFeasTol to 1e-09
   Prev: 1e-05  Min: 1e-09  Max: 0.1  Default: 1e-05


/usr/lib/python2.7/site-packages/ipykernel/__main__.py:11 DeprecationWarning: object of type <type 'float'> cannot be safely interpreted as an integer.


In [10]:
master.add_submodels(sub_dict)

In [11]:
from dynamicme.callback_gurobi import cb_benders_multi, cb_benders

In [12]:
#master.model.Params.Presolve = 0
#master.model.Params.ScaleFlag = 0
#master.model.Params.Heuristics = 0   # cbLazy might be bugged by heuristics

In [13]:
# master.model.getRow(master.model.getConstrByName('z_cut'))

# 1) Solve the LP relaxation

In [14]:
### Initial core point: y_jkl=1 for digit(k)=1, power(l)=0 forall j
nkeffs = len(var_cons_dict)

var_ind = {v:j for j,v in enumerate(master._ys)}
y0 = np.ones(len(master._ys))
for r in var_cons_dict.keys():
    for k in range(len(digits)):
        for l in range(len(powers)):
            var = master.model.getVarByName('binary_%s%s%s'%(r,k,l))
            ind = var_ind[var]
            if digits[k]==1 and powers[l]==0:                
                y0[ind] = 1
            else:
                y0[ind] = 0
            ### Also use as initial feasible solution (and possibly incumbent)
            var.Start = y0[ind]
sum(y0)



48.0

In [15]:
def opt_nondom(sub, y0):
    # Add or update the constraint for same obj
    cons = sub.model.model.getConstrByName('fixobjval')
    if cons is None:
        expr = sub.model.model.getObjective() == sub.model.model.ObjVal
        cons = sub.model.model.addConstr(expr, name='fixobjval')
    else:
        cons.RHS = sub.model.model.ObjVal
        obj = sub.model.model.getObjective()
        for j in range(obj.size()):
            v = obj.getVar(j)
            sub.model.model.chgCoeff(cons, v, obj.getCoeff(j))
    # Change the objective function
    sub.update_obj(y0)
    sub.model.optimize()
    # Relax the constraint for next iter
    cons.Sense = GRB.LESS_EQUAL
    cons.RHS   = GRB.INFINITY

In [16]:
from __future__ import division
import time
master.precision_sub = 'gurobi'
master.verbosity = 1
master.print_iter = 20

master.model.Params.OutputFlag = 0
master.model.Params.TimeLimit = 3600*2

### LP Relaxation
if LP_RELAXATION:
    print('Performing LP relaxation of Master')
    for v in master.model.getVars():
        v.VType = GRB.CONTINUOUS
###--------------
maxiter = 1000
UB = 1e15
LB =-1e15
gap= UB-LB
relgap = gap / (1e-10+abs(UB))  
tic = time.time()
for _iter in range(maxiter):
    if np.mod(_iter,master.print_iter)==0:
        toc = time.time()-tic
        print("iter=%d\tUB=%g\tLB=%g\tgap=%g\trelgap=%.4g%%\ttime=%g (s)" % (_iter,UB,LB,gap,relgap*100,toc))
        
    master.model.optimize()    
    fy = master._fy
    ys = master._ys
    z  = master._z
    yopt = [y.X for y in ys]
    sub_dict = master.sub_dict
    sub_objs = []
    opt_sub_inds = []
    
    for sub_ind,sub in iteritems(sub_dict):
        sub.update_obj(yopt)
        sub.model.optimize(precision=master.precision_sub)                
        if sub.model.Status==GRB.Status.UNBOUNDED:
            feascut = master.make_feascut(sub)
            master.model.addConstr(feascut)
        else:
            sub_obj = sub._weight*sub.model.ObjVal
            sub_objs.append(sub_obj)
            opt_sub_inds.append(sub_ind)
            ### Get non-dominated cut
            y0 = 0.5*(y0 + yopt)
            y0[y0<master.model.Params.IntFeasTol] = 0.
            opt_nondom(sub, y0)
            ###

    
    LB = z.X
    UB = sum(fy*yopt) + sum(sub_objs)
    gap= UB-LB
    relgap = gap / (1e-10+abs(UB))    
    
    if relgap < master.gaptol:        
        toc = time.time()-tic
        print("iter=%d\tUB=%g\tLB=%g\tgap=%g\trelgap=%.4g%%\ttime=%g (s)" % (_iter,UB,LB,gap,relgap*100,toc))
        print("gap<gaptol. Done.")
        break
    else:
        for sub_ind in opt_sub_inds:
            sub = sub_dict[sub_ind]
            cut = master.make_optcut(sub)
            master.model.addConstr(cut)

Performing LP relaxation of Master
iter=0	UB=1e+15	LB=-1e+15	gap=2e+15	relgap=200%	time=0.000471115 (s)
iter=20	UB=0.360925	LB=0	gap=0.360925	relgap=100%	time=1.06382 (s)
iter=40	UB=0.297496	LB=0	gap=0.297496	relgap=100%	time=2.0725 (s)
iter=60	UB=0.206259	LB=0	gap=0.206259	relgap=100%	time=3.08773 (s)
iter=80	UB=0.342197	LB=0	gap=0.342197	relgap=100%	time=4.10026 (s)
iter=100	UB=0.258931	LB=0	gap=0.258931	relgap=100%	time=5.11375 (s)
iter=120	UB=0.277713	LB=0	gap=0.277713	relgap=100%	time=6.12125 (s)
iter=140	UB=0.287447	LB=0	gap=0.287447	relgap=100%	time=7.14056 (s)
iter=160	UB=0.203418	LB=0	gap=0.203418	relgap=100%	time=8.15356 (s)
iter=180	UB=0.237486	LB=0	gap=0.237486	relgap=100%	time=9.16409 (s)
iter=200	UB=0.30375	LB=0	gap=0.30375	relgap=100%	time=10.1923 (s)
iter=220	UB=0.215815	LB=0	gap=0.215815	relgap=100%	time=11.2173 (s)
iter=240	UB=0.247448	LB=0	gap=0.247448	relgap=100%	time=12.2402 (s)
iter=260	UB=0.232155	LB=0	gap=0.232155	relgap=100%	time=13.2718 (s)
iter=280	UB=0.28188

# Using callback

master.precision_sub = 'gurobi'
master.verbosity = 1
master.print_iter = 1000

#master.model.Params.MIPGap = 0.001
master.model.Params.OutputFlag = 1
master.model.Params.TimeLimit = 3600*2
master.model.optimize(cb_benders_multi)
# master.model.optimize(cb_benders)

In [17]:
sol_master = {x.VarName:x.X for x in master.model.getVars()}

yopt = [sol_master[y.VarName] for y in master._ys]
print('Number of non-zero binaries: %g' % sum(yopt))
# Fitted parameters

kfit_dict = {}
for group_id, var_dict in iteritems(var_cons_dict):
    var = var_dict[0]
    cons = var_dict[1]
    a0  = var_dict[0][2]
    kfit = 0.
    for l,pwr in enumerate(powers):
        for k,digit in enumerate(digits):            
            yid = 'binary_%s%s%s' % (group_id,k,l)
            y   = sol_master[yid]
#             if abs(y)>1e-10:
#                 print('%s. Value=%s. Power=%g. Digit=%g' % (yid, y, pwr, digit))            
            kfit += y*a0*radix**pwr*digit
    kfit_dict[group_id] = kfit

kfit_changed = [(k,v, abs(v-a0)/a0) for k,v in iteritems(kfit_dict) if abs(v-a0)/a0>1e-6]
print('Changed keffs: %d/%d' % (len(kfit_changed), len(var_cons_dict)))
# print('kfit_changed:',kfit_changed)

#----------------------------------------
# Starting from basal model
csrcs = df_conds.cond.unique()
for csrc in csrcs:    
    ijofit = load_json_model('/home/laurence/ME/models/e_coli_core_pc.json')
    crowding = ijofit.metabolites.get_by_id('crowding')
    df_condi = df_conds[ df_conds.cond==csrc]    
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.lower_bound = row['lb']
        rxn.upper_bound = row['ub']

    for rid,kfit in iteritems(kfit_dict):
        rxn = ijofit.reactions.get_by_id(rid)
        rxn.add_metabolites({crowding:kfit}, combine=False)
    
    ijofit.optimize()
    
    mu_measi = df_meas[ df_meas.substrate==csrc].growth_rate_1_h.iloc[0]
    mu_fiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    
    # Get unfit
    for rxn in ijofit.metabolites.crowding.reactions:
        rxn._metabolites[crowding] = a0
    ijofit.optimize()
    mu_unfiti = ijofit.reactions.BIOMASS_Ecoli_core_w_GAM.x
    err0= 100*(mu_unfiti-mu_measi)/mu_measi
    err = 100*(mu_fiti - mu_measi)/mu_measi
    derr= 100*(abs(err)-abs(err0))/abs(err0)
    print('Cond=%s. mu_meas=%g. mu_sim=%g (unfit=%g, error=%.3g%%). Error=%.3g%% (%.3g%% change)' % (csrc, mu_measi, mu_fiti, mu_unfiti, err0, err, derr))
    for i,row in df_condi.iterrows():
        rid = row['rxn']
        rxn = ijofit.reactions.get_by_id(rid)        
        print('\t%s uptake=%g' % (rxn.id, rxn.x))

Number of non-zero binaries: 75.6486
Changed keffs: 45/48
Cond=glucose. mu_meas=0.74. mu_sim=0.72253 (unfit=0.873922, error=18.1%). Error=-2.36% (-87% change)
	EX_glc__D_e uptake=-10
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=0
Cond=acetate. mu_meas=0.256. mu_sim=0.307686 (unfit=0.389313, error=52.1%). Error=20.2% (-61.2% change)
	EX_glc__D_e uptake=0
	EX_fru_e uptake=0
	EX_succ_e uptake=0
	EX_mal__L_e uptake=0
	EX_ac_e uptake=-20


In [18]:
kfit_changed

[(u'G6PDH2r', 6.410256410256411e-06, 0.5),
 (u'ACKr', 3.846153846153847e-06, 0.09999999999999992),
 (u'MALS', 1.2820512820512822e-05, 2.0),
 (u'GLUN', 1.9230769230769234e-06, 0.5499999999999999),
 (u'ME1', 1.2820512820512822e-05, 2.0),
 (u'GND', 1.0683760683760684e-05, 1.4999999999999998),
 (u'ME2', 1.9230769230769234e-06, 0.5499999999999999),
 (u'ADK1', 6.410256410256411e-06, 0.5),
 (u'SUCDi', 1.0683760683760684e-05, 1.4999999999999998),
 (u'PPC', 2.136752136752137e-06, 0.5),
 (u'MDH', 1.495726495726496e-05, 2.5),
 (u'FUM', 1.0683760683760684e-05, 1.4999999999999998),
 (u'GLUDy', 1.9230769230769234e-06, 0.5499999999999999),
 (u'GLUSy', 1.9230769230769234e-06, 0.5499999999999999),
 (u'PGL', 1.1336581928685882e-05, 1.6527601713124962),
 (u'PGM', 2.2807140473696374e-06, 0.46631291291550486),
 (u'ACALD', 1.9230769230769234e-06, 0.5499999999999999),
 (u'PGK', 4.8589566934137425e-06, 0.13699586625881566),
 (u'PGI', 8.471670746433526e-06, 0.9823709546654449),
 (u'PPS', 2.136752136752137e-06,